In [1]:
import logging
from IPython.core.interactiveshell import InteractiveShell
%load_ext autoreload
InteractiveShell.ast_node_interactivity = "all"

logging.basicConfig(
    level=logging.DEBUG, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

In [3]:
import sys
import os

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [102]:
%autoreload 2
import logging
import os

import time
import fire

import pandas as pd
from datetime import datetime
from utils.ims_utils import (
    load_dotd_data,
    export_im_and_ms1scans,
    combine_3d_act_and_sum_int,
)
from utils.constants import exp_cols
from utils.config import get_cfg_defaults
from utils.singleton_swaps_optimization import swaps_optimization_cfg
from optimization.inference import process_ims_frames_parallel, generate_id_partitions
from result_analysis import result_analysis

# from result_analysis import result_analysis
from prepare_dict.prepare_dict import construct_dict

# from result_analysis import result_analysis
config_path = "/cmnfs/proj/ORIGINS/SWAPS_exp/tims_ramp_time/corrected_RT_tol_pred_120min_library_80ms_20240729_105248_996985/config_test_construct_dict.yaml"
logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    level=logging.INFO,
)

cfg = get_cfg_defaults(swaps_optimization_cfg)
name_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
if config_path is not None:
    cfg.merge_from_file(config_path)
    logging.info("merge with cfg file %s", config_path)
if cfg.ADD_TIMESTAMP_TO_RESULT_PATH:
    cfg.RESULT_PATH = cfg.RESULT_PATH + "_" + name_timestamp
    cfg.ADD_TIMESTAMP_TO_RESULT_PATH = False  # in case of reuse of config file
logging.info("==================Load data==================")
os.makedirs(cfg.RESULT_PATH, exist_ok=True)

# Load data
data, hdf_file_name = load_dotd_data(
    cfg.DATA_PATH, swaps_result_dir=cfg.EXPORT_DATA_HDF5_DIR
)
maxquant_result_exp = pd.read_csv(cfg.MQ_EXP_PATH, sep="\t", low_memory=False)
# Get the lowest level directory name with .d extension
dir_with_extension = os.path.basename(os.path.normpath(cfg.DATA_PATH))
if (
    cfg.FILTER_EXP_BY_RAW_FILE == ""
):  # if not specified, get the lowest level directory name with .d extension, by default None
    cfg.FILTER_EXP_BY_RAW_FILE = dir_with_extension.rstrip(".d")

maxquant_result_exp = maxquant_result_exp.loc[
    maxquant_result_exp["Raw file"] == cfg.FILTER_EXP_BY_RAW_FILE,
    :,
]
maxquant_result_ref = pd.read_csv(cfg.MQ_REF_PATH, sep="\t", low_memory=False)
# TODO filter ref df if needed

ms1scans, mobility_values_df = export_im_and_ms1scans(
    data=data, swaps_result_dir=cfg.RESULT_PATH
)
if cfg.DICT_PICKLE_PATH != "":
    maxquant_result_ref = pd.read_pickle(filepath_or_buffer=cfg.DICT_PICKLE_PATH)
else:
    maxquant_result_ref, dict_pickle_path, cfg_prepare_dict = construct_dict(
        cfg_prepare_dict=cfg.PREPARE_DICT,
        maxquant_exp_df=maxquant_result_exp,
        maxquant_ref_df=maxquant_result_ref,
        result_dir=os.path.join(cfg.RESULT_PATH),
        mobility_values_df=mobility_values_df,
        rt_values_df=ms1scans,
        random_seed=cfg.RANDOM_SEED,
    )
    cfg.PREPARE_DICT = cfg_prepare_dict
    cfg.DICT_PICKLE_PATH = dict_pickle_path
cfg.dump(
    stream=open(
        os.path.join(cfg.RESULT_PATH, f"config_{name_timestamp}.yaml"),
        "w",
        encoding="utf-8",
    )
)
logging.info(
    "Finished dictionary preparation and saved config to %s",
    os.path.join(cfg.RESULT_PATH, f"config_{name_timestamp}.yaml"),
)

2024-08-05 13:38:31,286 - root - INFO - merge with cfg file /cmnfs/proj/ORIGINS/SWAPS_exp/tims_ramp_time/corrected_RT_tol_pred_120min_library_80ms_20240729_105248_996985/config_test_construct_dict.yaml
2024-08-05 13:38:31,287 - root - INFO - ==================Load data==================
2024-08-05 13:38:31,289 - root - INFO - Importing data from /cmnfs/proj/ORIGINS/data/tims_ramp_time/Hela2ug_lowflow_30min_1to37to42_NCE29to59_80ms7R_RA2_1_2075.d
2024-08-05 13:38:31,293 - root - INFO - Using HDF import for /cmnfs/proj/ORIGINS/data/tims_ramp_time/Hela2ug_lowflow_30min_1to37to42_NCE29to59_80ms7R_RA2_1_2075.hdf


2024-08-05 13:38:31,660 - root - INFO - Successfully imported data from /cmnfs/proj/ORIGINS/data/tims_ramp_time/Hela2ug_lowflow_30min_1to37to42_NCE29to59_80ms7R_RA2_1_2075.d
2024-08-05 13:38:31,662 - utils.ims_utils - INFO - No output directory provided, using the directory of the .d file
2024-08-05 13:38:31,664 - utils.ims_utils - INFO - HDF file /cmnfs/proj/ORIGINS/data/tims_ramp_time/Hela2ug_lowflow_30min_1to37to42_NCE29to59_80ms7R_RA2_1_2075.d/Hela2ug_lowflow_30min_1to37to42_NCE29to59_80ms7R_RA2_1_2075.hdf already exists
2024-08-05 13:38:39,892 - utils.ims_utils - INFO - Double check MS1 frame index range: 0 - 2961
2024-08-05 13:38:39,894 - utils.ims_utils - INFO - Double check mobility values index range: 0 - 749
2024-08-05 13:38:39,971 - prepare_dict.prepare_dict - INFO - RT index range: (0.0, 30.00002711666667)
2024-08-05 13:38:39,972 - prepare_dict.prepare_dict - INFO - IM index range: (0.8499903284315748, 1.3215078344250004)
2024-08-05 13:38:39,974 - prepare_dict.prepare_dict 

30/30 [==============================] - 4s 121ms/step


2024-08-05 13:39:23,283 - deeplc.deeplc - DEBUG - Selecting the data points for calibration (used to fit the linear models between)
2024-08-05 13:39:23,284 - deeplc.deeplc - DEBUG - Skipping calibration step, due to no points in the predicted range (are you sure about the split size?): 0.0,0.556383285522461
2024-08-05 13:39:23,285 - deeplc.deeplc - DEBUG - Skipping calibration step, due to no points in the predicted range (are you sure about the split size?): 0.556383285522461,1.112766571044922
2024-08-05 13:39:23,291 - deeplc.deeplc - DEBUG - Fitting the linear models between the points
2024-08-05 13:39:24,562 - deeplc.deeplc - DEBUG - Extracting features for the CNN model ...
2024-08-05 13:39:24,563 - deeplc.deeplc - DEBUG - prepare feature extraction
2024-08-05 13:39:24,564 - deeplc.deeplc - DEBUG - start feature extraction
2024-08-05 13:39:24,565 - deeplc.feat_extractor - DEBUG - Extracting CNN features
2024-08-05 13:39:28,613 - deeplc.feat_extractor - DEBUG - Time to calculate all

30/30 [==============================] - 4s 118ms/step


2024-08-05 13:39:34,795 - deeplc.deeplc - DEBUG - For full_hc_deeplc_train model got a performance of: 0.3444632484963464
2024-08-05 13:39:34,803 - deeplc.deeplc - DEBUG - Model with the best performance got selected: {'/cmnfs/proj/ORIGINS/SWAPS_exp/tims_ramp_time/corrected_RT_tol_pred_120min_library_80ms_20240729_105248_996985/construct_dict/RT_transfer_learn/full_hc_deeplc_train_8c22d89667368f2f02ad996469ba157e.hdf5': '/cmnfs/proj/ORIGINS/SWAPS_exp/tims_ramp_time/corrected_RT_tol_pred_120min_library_80ms_20240729_105248_996985/construct_dict/RT_transfer_learn/full_hc_deeplc_train_8c22d89667368f2f02ad996469ba157e.hdf5'}
2024-08-05 13:39:51,035 - prepare_dict.predict_rt - INFO - pred_evidence_transfer_file is /cmnfs/proj/ORIGINS/SWAPS_exp/tims_ramp_time/corrected_RT_tol_pred_120min_library_80ms_20240729_105248_996985/construct_dict/pred_transfer.txt
2024-08-05 13:41:27,333 - deeplc.deeplc - DEBUG - Extracting features for the CNN model ...
2024-08-05 13:41:27,334 - deeplc.deeplc - DEBU

162/162 [==============================] - 20s 118ms/step


2024-08-05 13:42:39,153 - prepare_dict.prepare_dict - DEBUG - dict_add_rt_index columns: Index(['Sequence', 'Length', 'Modifications', 'Modified sequence',
       'Oxidation (M) Probabilities', 'Oxidation (M) Score Diffs',
       'Acetyl (Protein N-term)', 'Oxidation (M)', 'Missed cleavages',
       'Proteins',
       ...
       'IM_search_idx_center', 'RT_search_left', 'RT_search_right',
       'RT_search_center', 'Time_minute_center_ref',
       'MS1_frame_idx_center_ref', 'Time_minute_left_ref',
       'MS1_frame_idx_left_ref', 'Time_minute_right_ref',
       'MS1_frame_idx_right_ref'],
      dtype='object', length=115)
2024-08-05 13:43:17,766 - root - INFO - Finish. Filtered prediction dataframe dimension: (179460, 120), columns: Index(['Sequence', 'Length', 'Modifications', 'Modified sequence',
       'Oxidation (M) Probabilities', 'Oxidation (M) Score Diffs',
       'Acetyl (Protein N-term)', 'Oxidation (M)', 'Missed cleavages',
       'Proteins',
       ...
       'MS1_frame_idx